# Tellurify
**Objective: Convert an input PDB volume into a Telluroienylalanine-substituted PDB**

**Algorithm**
1. Read groups of phenylalanine ATOM records from a PDB.
2. For each group, get CB, CG, CB->CG vector, and dihedral angle CA-CB:CG:CD1
3. Delete all atoms after CB, insert tellurienyl ring and calculate the new dihedral
4. Rotate the tellurienyl ring about the CB:CG axis until the dihedral is as desired.

Need to think about how we're going to encode the tellurienyl ring.
We want something where we can just offset the coordinates of the atoms by the coordinates of CB and have it 

When we insert the ring, we will be creating the BG bond.
This bond needs to be a vector parallel and with the same origin as the old BG bond.

1. Define a rigid body CB-CG-CD-CE-CZ-Te-(CG) (the ring plus a stub)
2. Find a rotation (axis must be through CB/origin) that translates vector BG to have the same slope of old vector BG (same 3d slope).
3. Find a translation that moves CB onto initial CB

https://www.euclideanspace.com/maths/geometry/rotations/conversions/angleToQuaternion/index.htm

https://stackoverflow.com/questions/4870393/rotating-coordinate-system-via-a-quaternion

https://stackoverflow.com/questions/1171849/finding-quaternion-representing-the-rotation-from-one-vector-to-another


https://bitbucket.org/sinbad/ogre/src/9db75e3ba05c/OgreMain/include/OgreVector3.h#cl-651

In [164]:
import sys
import copy
import math

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from Bio.PDB import *
import nglview as nv # molecular graphics viewing system !?

%matplotlib notebook
mpl.rcParams['figure.dpi'] = 100
plt.style.use(['dark_background'])

In [197]:
def coordsOfAtomFromResidue(atomName, resi):
    return [a for a in resi.get_atoms() if a.get_name() == atomName][0].get_coord()

def quatConj(q):
    # Compute the conjugate of quaternion q
    return [q[0], -1*q[1], -1*q[2], -1*q[3]]

def quatMult(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 + y1 * w2 + z1 * x2 - x1 * z2
    z = w1 * z2 + z1 * w2 + x1 * y2 - y1 * x2
    return w, x, y, z

def quatToRotateAontoB(a,b):
    theta = np.arccos(np.dot(a,b))
    axis = normalize(np.cross(a, b))
    s = np.sin(theta/2)
    q = [np.cos(theta/2), axis[0]*s, axis[1]*s, axis[2]*s]
    q = q / np.linalg.norm(q)
    return q

def transformVectorByQuaternion(v, q):
    # Transform a 3d vector by quaternion q, and return a 3D vector
    vq = [0, v[0], v[1], v[2]]
    transformed = quatMult(quatMult(q, vq), quatConj(q))
    
    return [transformed[1], transformed[2], transformed[3]]
    
def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm

def bruteForceAlignRingOrientation(tephe0, phe, offset=0):
     # Get plane normals by crossproduct of two vectors in each ring
    pGD1 = normalize(np.subtract(phe['CD1'], phe['CG']))
    pGD2 = normalize(np.subtract(phe['CD2'], phe['CG']))
    tGD = normalize(np.subtract(tephe0['CD'], tephe0['CG']))
    tGTe = normalize(np.subtract(tephe0['Te'], tephe0['CG']))
    
    tBG = normalize(np.subtract(tephe0['CB'],tephe0['CG']))
    tGZ = normalize(np.subtract(tephe0['CZ'], tephe0['CG']))
    pGZ = normalize(np.subtract(phe['CZ'], phe['CG']))
    phe_norm = normalize(np.cross(pGD1, pGD2))
    tephe_norm = normalize(np.cross(tGD, tGTe))
    
    ring_delta = np.arccos(np.dot(tephe_norm, phe_norm))    
    
    thetas = []
    deltas = []
    tephes = []

    for theta in np.linspace(0, 360, num=3600):
        tephe = copy.deepcopy(tephe0)
        
        # Generate the rotation
        theta = theta*np.pi/180
        axis = normalize(pGZ)
        s = np.sin(theta/2)
        q = [np.cos(theta/2), axis[0]*s, axis[1]*s, axis[2]*s]
        q = q / np.linalg.norm(q)

        # Apply the quaternion rotation to the tephe points
        for key,value in tephe.items():
            tephe[key] = transformVectorByQuaternion(value, q)
        
        ## MEASURE THE RING ANGLE AGAIN
        pGD1 = normalize(np.subtract(phe['CD1'], phe['CG']))
        pGD2 = normalize(np.subtract(phe['CD2'], phe['CG']))

        tGD = normalize(np.subtract(tephe['CD'], tephe['CG']))
        tGTe = normalize(np.subtract(tephe['Te'], tephe['CG']))

        tBG = normalize(np.subtract(tephe['CG'],tephe['CB']))

        phe_norm = normalize(np.cross(pGD1, pGD2))
        tephe_norm = normalize(np.cross(tGD, tGTe))

        ring_delta = np.arccos(np.dot(tephe_norm, phe_norm))
        
        thetas.append(theta) # input values (how much the ring was rotated)
        deltas.append(ring_delta) # orientation difference between rings
        tephes.append(tephe) # the actual coordinates
    deltas = np.asarray(deltas)
    idx = (np.abs(deltas - offset)).argmin()
    
    return tephes[idx], deltas[idx]

In [201]:
# Import data from the PDB
parser = PDBParser()
structure = parser.get_structure('vrk1', '6ac9.pdb')

F = [r for r in structure[0]['A'] if r.get_resname() == "PHE"]

print("Loaded PDB and found %d phenylalanines." % len(F))

# Get the TePhe rigid body
tephe_structure = parser.get_structure('tephe', 'tephe_rectified.pdb')
tephe_resi = list(tephe_structure[0].get_residues())[0]

# Make a dictionary with all the necessary TePhe atoms (ring plus beta carbon)
tephe0 = {
    'CB' : coordsOfAtomFromResidue('CB',tephe_resi),
    'CG' : coordsOfAtomFromResidue('CG',tephe_resi),
    'CD' : coordsOfAtomFromResidue('CD',tephe_resi),
    'CE' : coordsOfAtomFromResidue('CE',tephe_resi),
    'CZ' : coordsOfAtomFromResidue('CZ',tephe_resi),
    'Te' : coordsOfAtomFromResidue('Te',tephe_resi)
}

# Keep track of the coordinates for each TePHe
tephes = []
phes = []

# Iterate through the phenylalanines
for i, f in enumerate(F):    
    print(f.get_id()[1])
    tephe = copy.deepcopy(tephe0)
    # Get the PHE atomic coordinates needed to position the new TEPHE ring.
    phe = {
        'CA' : coordsOfAtomFromResidue('CA',f),
        'CB' : coordsOfAtomFromResidue('CB',f),
        'CG' : coordsOfAtomFromResidue('CG',f),
        'CD1' : coordsOfAtomFromResidue('CD1',f),
        'CD2' : coordsOfAtomFromResidue('CD2',f),
        'CE1' : coordsOfAtomFromResidue('CE1',f),
        'CE2' : coordsOfAtomFromResidue('CE2',f),
        'CZ' : coordsOfAtomFromResidue('CZ',f)
    }
    
    phes.append(phe)
    
    # STEP 1: ALIGN THE STEM VECTORS
    # Find the stem vectors between beta and gamma carbons
    tBG = normalize(np.subtract(tephe['CG'],tephe['CB']))
    pBG = normalize(np.subtract(phe['CG'],phe['CB']))
    
    # Calculate the quaternion rotation that makes tBG parallel to pBG
    q = quatToRotateAontoB(tBG, pBG)

    # Apply the quaternion rotation to the tephe points
    for key,value in tephe.items():
        tephe[key] = transformVectorByQuaternion(value, q)
    
    # STEP 2: ALIGN THE RING PLANES
    # Using a brute force approach. It is unclear why the principled approach wasn't working.
    tephe, ring_delta = bruteForceAlignRingOrientation(tephe, phe, 0)
    
    print("Ring Delta for %d: %0.2f" % (i,ring_delta*(180/np.pi)))
        
    # Translate the ring so that tCB aligns with pCB
    translation = np.subtract(phe['CB'],tephe['CB'])
    for key,value in tephe.items():
        tephe[key] = np.add(value, translation)
        
    # Construct the TePhe residue (N, CA, C, O, CB, CG, CD, CE, CZ, Te)
    f.resname = 'TPE'
    
    # Remove the PHE atoms after CA from the PDB file. Also set the position of CB.
    toDelete = []
    for atom in f.get_atoms():
        if atom.get_name() in ['CD1', 'CD2', 'CE1', 'CE2', 'CZ']:
            toDelete.append(atom.get_id())
        elif atom.get_name() == 'CB':
            atom.set_coord(tephe['CB'])
        elif atom.get_name() == 'CG':
            atom.set_coord(tephe['CG'])

    for i in toDelete:
        f.detach_child(i)
#               name, coord, bf, occ, altloc, fullname, serial, element
    cd = Atom.Atom('CD', tephe['CD'], 0, 1, ' ', 'CD', 'C', 'C')
    ce = Atom.Atom('CE', tephe['CE'], 0, 1, ' ', 'CE', 'C', 'C')
    cz = Atom.Atom('CZ', tephe['CZ'], 0, 1, ' ', 'CZ', 'C', 'C')
    te = Atom.Atom('Te', tephe['Te'], 0, 1, ' ', 'Te', 'C', 'TE')

    f.add(cd)
    f.add(ce)
    f.add(cz)
    f.add(te)
    
    tephes.append(copy.deepcopy(tephe))
    
io = PDBIO()
io.set_structure(structure)
io.save('out.pdb')


Loaded PDB and found 8 phenylalanines.
23
Ring Delta for 0: 0.26
81
Ring Delta for 1: 0.18
86
Ring Delta for 2: 2.62
128
Ring Delta for 3: 1.69
134
Ring Delta for 4: 6.26
149
Ring Delta for 5: 1.33
227
Ring Delta for 6: 1.72
290
Ring Delta for 7: 2.15


In [136]:
def plotDictionaryOnAxes(ax, d, color):
    xs = [v[0] for k,v in d.items()]
    ys = [v[1] for k,v in d.items()]
    zs = [v[2] for k,v in d.items()]
    ax.scatter(xs, ys, zs, color=color)

from mpl_toolkits.mplot3d import Axes3D  
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection="3d")

# for t in tephes:
#     plotDictionaryOnAxes(ax, t,"b")
# for t in phes:
#     plotDictionaryOnAxes(ax, t,"r")
i = 2
# plotDictionaryOnAxes(ax, phes[i], "r")
# plotDictionaryOnAxes(ax, tephes[i], "b")
plotDictionaryOnAxes(ax, tephe0, "r")
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

<IPython.core.display.Javascript object>

In [112]:
tephe0
np.arccos(np.dot(normalize(np.subtract(tephe0['CG'],tephe0['CZ'])),normalize(np.subtract(tephe0['CG'],tephe0['CD']))))*(180/np.pi)

61.16419375330244

IndentationError: unexpected indent (<ipython-input-11-ad2ceabd02bf>, line 2)